In [421]:
import json
from pymongo import MongoClient
import random 
from datetime import datetime
from bson import ObjectId
import requests
import pandas as pd

In [422]:
dbName = "dbChat"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()
#cur = db.list_collections()


mongodb://localhost/dbChat


In [423]:
path='../quotes-100-en.json'
with open(path,'r') as file:
    data=json.load(file)



In [424]:
users=list(data.keys())
messages=list(data.values())
chat_ids=[1,2,3,4,5,6,7,8,9]

In [ ]:
'''#build the list of dictionaries for each message
list_of_messages=[]
for i,message in enumerate(messages):
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'user_id':random.choice(users),
         'text':message,
         'time':dt_string,
         'chat_id':[],
    
     }
    list_of_messages.append(dic)
#messages_ids=db.messages.insert_many(list_of_messages)'''

In [425]:
#build the list of dictionaries for users
list_of_users=[]
for i,user in enumerate(users):
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'user_name':random.choice(users),
         'insertion_date':dt_string,
         'chats_list':[]
    
         }
    list_of_users.append(dic)
users_ids=db.users.insert_many(list_of_users)

In [426]:
#build the list of dictionaries for chats
list_of_chats=[]
for chat in chat_ids:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'chat_name':random.choice(chat_ids),
         'creation_date':dt_string,
         'users_list':[random.choice(list(users_ids.inserted_ids)) for i in range(50)],
         'messages_list':[]
    
     }
    list_of_chats.append(dic)
    
chats_ids=db.chats.insert_many(list_of_chats)

In [427]:
##populate the database with the messages with the api

messages_ids=[]
for chat_id in chats_ids.inserted_ids:
    for i in range(90):#add 10 messages
        chat_id=str(chat_id)
        user=random.choice(list(users_ids.inserted_ids))
        #add the user to the chat
        r = requests.get(f'http://localhost:3500/chat/{chat_id}/adduser?user_id={user}')
       
        text=random.choice(messages)
        r = requests.get(f'http://localhost:3500/chat/{chat_id}/addmessage?user_id={user}&text={text}')
        
        messages_ids.append(r)
        

In [432]:
chats_ids.inserted_ids

[ObjectId('5eacf49d01e77d744d478a0d'),
 ObjectId('5eacf49d01e77d744d478a0e'),
 ObjectId('5eacf49d01e77d744d478a0f'),
 ObjectId('5eacf49d01e77d744d478a10'),
 ObjectId('5eacf49d01e77d744d478a11'),
 ObjectId('5eacf49d01e77d744d478a12'),
 ObjectId('5eacf49d01e77d744d478a13'),
 ObjectId('5eacf49d01e77d744d478a14'),
 ObjectId('5eacf49d01e77d744d478a15')]

In [430]:
users_ids.inserted_ids


[ObjectId('5eacf48701e77d744d476d65'),
 ObjectId('5eacf48701e77d744d476d66'),
 ObjectId('5eacf48701e77d744d476d67'),
 ObjectId('5eacf48701e77d744d476d68'),
 ObjectId('5eacf48701e77d744d476d69'),
 ObjectId('5eacf48701e77d744d476d6a'),
 ObjectId('5eacf48701e77d744d476d6b'),
 ObjectId('5eacf48701e77d744d476d6c'),
 ObjectId('5eacf48701e77d744d476d6d'),
 ObjectId('5eacf48701e77d744d476d6e'),
 ObjectId('5eacf48701e77d744d476d6f'),
 ObjectId('5eacf48701e77d744d476d70'),
 ObjectId('5eacf48701e77d744d476d71'),
 ObjectId('5eacf48701e77d744d476d72'),
 ObjectId('5eacf48701e77d744d476d73'),
 ObjectId('5eacf48701e77d744d476d74'),
 ObjectId('5eacf48701e77d744d476d75'),
 ObjectId('5eacf48701e77d744d476d76'),
 ObjectId('5eacf48701e77d744d476d77'),
 ObjectId('5eacf48701e77d744d476d78'),
 ObjectId('5eacf48701e77d744d476d79'),
 ObjectId('5eacf48701e77d744d476d7a'),
 ObjectId('5eacf48701e77d744d476d7b'),
 ObjectId('5eacf48701e77d744d476d7c'),
 ObjectId('5eacf48701e77d744d476d7d'),
 ObjectId('5eacf48701e77d

# Nltk analysis

In [ ]:
!pip3 install nltk

In [435]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/silviaserafini/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [295]:
sia = SentimentIntensityAnalyzer()

In [491]:
import ast
chat_id='5eacf49d01e77d744d478a0e'
r = requests.get(f'http://localhost:3500//chat/{chat_id}/list')
messages1=r.json()
#print(messages1)

users_messages={}
#prendo tutti i messaggi di un utente e li unisco
for k,v in messages1.items():
    mes_user=db.messages.find_one({'_id':ObjectId(k)})#fetcg=h the message
    user=mes_user['user_id']#fetch the user
    user_texts=list(db.messages.find({"user_id":user},{'text':1}))#fetch all the messages for the user
    user_text=''
    for text in user_texts:
        text = ast.literal_eval(text['text'])
        txt=''
        for t in text:#all the messages in an unic string
            txt += t
        user_text=user_text +' '+ txt
        users_messages[user]=user_text   
    
    
sentiments={}#k =user, value={text,score}
for k,v in users_messages.items():    
    sentiments[k]={'text':v,"score":sia.polarity_scores(v)}
sent1={}
for k,v in sentiments.items():#k =user, value=text
    sent1[k]=v['text']

In [493]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

sparse_matrix = count_vectorizer.fit_transform(sent1.values())

In [494]:
sparse_matrix
text_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(text_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=sent1.keys())
display(df)

,105,122,145ffthe,14th,17,1984,1997,1997these,20,205,...,yee,yesterday,yet,you,young,your,yourself,youth,zeal,zero
5eacf48701e77d744d4778d4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5eacf48701e77d744d477fa4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5eacf48701e77d744d4782cf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
5eacf48701e77d744d477de2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5eacf48701e77d744d47813a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5eacf48701e77d744d478704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5eacf48701e77d744d4778c0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5eacf48701e77d744d478116,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5eacf48701e77d744d47733b,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
set(stopwords.words('english'))